In [15]:
import os

In [13]:
%pwd

'd:\\COLLEGE\\first\\PROJECT\\GOOD\\Kidney-Disease-Classification'

In [12]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\envs\kidney\python.exe


In [3]:
os.chdir("../")

In [14]:
%pwd

'd:\\COLLEGE\\first\\PROJECT\\GOOD\\Kidney-Disease-Classification'

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [18]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "CT-KIDNEY-DATASET-Normal-Stone"
        )

        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )


In [19]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [20]:
from tensorflow.keras.applications.vgg16 import preprocess_input
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            preprocessing_function=preprocess_input,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=10,      
                horizontal_flip=True,
                zoom_range=0.1,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=validation_steps
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [21]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()

    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2025-12-27 15:40:24,540: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-27 15:40:24,555: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-27 15:40:24,557: INFO: common: created directory at: artifacts]
[2025-12-27 15:40:24,559: INFO: common: created directory at: artifacts\training]
Found 770 images belonging to 2 classes.
Found 3084 images belonging to 2 classes.
Epoch 1/10
192/192 [==============================] - 541s 3s/step - loss: 164.0553 - accuracy: 0.8677 - val_loss: 40.5247 - val_accuracy: 0.8633
Epoch 2/10
192/192 [==============================] - 565s 3s/step - loss: 4.7010 - accuracy: 0.9844 - val_loss: 24.6503 - val_accuracy: 0.9206
Epoch 3/10
192/192 [==============================] - 596s 3s/step - loss: 11.1210 - accuracy: 0.9795 - val_loss: 16.7215 - val_accuracy: 0.9492
Epoch 4/10
192/192 [==============================] - 546s 3s/step - loss: 2.0138 - accuracy: 0.9928 - val_loss: 106.0834 - val_accuracy: 0.7930
Epoch 5